In [8]:
!pip install --upgrade pip  
!pip install tweet-preprocessor
!pip install emot
!pip install textblob
import pandas as pd
import numpy as np
import warnings
import nltk
import preprocessor as prepr
import re
import emot
import textblob
import pickle
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from textblob import TextBlob
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords     
from multiprocessing import Pool
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO, EMOJI_UNICODE
import nltk
from sklearn import model_selection, naive_bayes, svm
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

warnings.filterwarnings('ignore')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

# !pip3 install xgboost
# !pip install xgboost
# import xgboost 
# !pip3 install catboost
# import catboost 
# from xgboost import XGBClassifier
# from catboost import CatBoostClassifier


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/akzanuali/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/akzanuali/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/akzanuali/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [11]:
df = pd.read_csv("train-balanced-sarcasm.csv")
data = df[['label', 'comment', 'parent_comment']]

In [15]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/akzanuali/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [19]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

stop_words = stopwords.words('english') + ['.',',','"','*']
lemmatizer = nltk.stem.WordNetLemmatizer()
w_tokenizer = TweetTokenizer()
# prepr.set_options(prepr.OPT.URL, prepr.OPT.EMOJI, prepr.OPT.MENTION, prepr.OPT.HASHTAG)
prepr.set_options(prepr.OPT.URL, prepr.OPT.MENTION, prepr.OPT.HASHTAG)

def convert_emojis(text):
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
    return text
    
def convert_emoticons(text):
    for emot in EMOTICONS_EMO:
        text = text.replace(emot, "_".join(EMOTICONS_EMO[emot].replace(",","").replace(":","").split()))
    return text
        
def clean_text(text):
    text = str(text)
#     text = str(TextBlob(text).correct())                                      #correcting spelling
#     text = emoji_pattern.sub(r'', text)                                     #removing emoji 
    text = prepr.clean(text)                                                  #removing links+mention+hashtags
    text = text.strip().lower()                                               #lowercase
    text = re.sub(r'<.*?>', '', text, flags=re.MULTILINE)                     #removing tags
    text = convert_emojis(text)                                               #converting emojis to text
    text = convert_emoticons(text)                
    text = [(lemmatizer.lemmatize(w)) for w in  w_tokenizer.tokenize((text))] #lemmatize+tokenization
    text = [item for item in text if item not in stop_words]                  #stopwords
    
    return text

In [20]:
texts = data.comment
cleaned_texts = []
for item in tqdm(texts, total = len(texts)):
    cleaned_texts.append(clean_text(item))

  0%|          | 0/1010826 [00:00<?, ?it/s]

In [21]:
final_texts = []
for text in tqdm(cleaned_texts, total = len(cleaned_texts)):
    final_texts.append(convert_emoticons(' '.join(text)))

  0%|          | 0/1010826 [00:00<?, ?it/s]

In [43]:
from collections import Counter
cnt = Counter()
for text in cleaned_texts:
    for word in text:
        cnt[word] += 1
        
cnt.most_common(10)

[('?', 118049),
 ('!', 97521),
 ('...', 79067),
 ('like', 56427),
 ('wa', 55805),
 ('yeah', 41086),
 ('get', 38664),
 ('people', 36412),
 ('would', 35187),
 ('one', 33950)]

In [22]:
X_train, X_test, y_train, y_test = train_test_split(final_texts, data['label'].values, test_size=0.3, random_state=0)

In [23]:
from sklearn import metrics  
def metrics_result(actual, predict):                       
    print('Metrics:')
    print ('Accuracy: {0: .3f}'.format(metrics.accuracy_score(actual, predict)) ) 
    print ('Precision: {0: .3f}'.format(metrics.precision_score(actual, predict,average='weighted')) ) 
    print ('Recall: {0: 0.3f}'.format(metrics.recall_score(actual, predict,average='weighted'))   )
    print ('F1-score: {0:.3f}'.format(metrics.f1_score(actual, predict,average='weighted'))   )

In [17]:
#TF Idf + Random Forest
text_clf = Pipeline([
                     ('tfidf', TfidfVectorizer()),
                     ('clf', RandomForestClassifier())
                     ])
 
text_clf.fit(X_train, y_train)
print('TF Idf + Random Forest:')
print()
predicted = text_clf.predict(X_test)
metrics_result(y_test, predicted)
print()

TF Idf + Random Forest:

Metrics:
Accuracy:  0.683
Precision:  0.683
Recall:  0.683
F1-score: 0.682



In [18]:
text_logreg = Pipeline([
                     ('tfidf', TfidfVectorizer()),
                     ('logreg', LogisticRegression())
                     ])
 
text_logreg.fit(X_train, y_train)

print('TF Idf + Logistic Regression:')
print()
predicted_l = text_logreg.predict(X_test)
metrics_result(y_test,predicted_l)
print()

TF Idf + Logistic Regression:

Metrics:
Accuracy:  0.680
Precision:  0.681
Recall:  0.680
F1-score: 0.679



In [19]:
text_nb = Pipeline([
                     ('tfidf', TfidfVectorizer()),
                     ('nb', naive_bayes.MultinomialNB())
                     ])
 
text_nb.fit(X_train, y_train)

print('TF Idf + Naive Bayes:')
print()
predicted_nb = text_nb.predict(X_test)
metrics_result(y_test,predicted_nb)
print()

TF Idf + Naive Bayes:

Metrics:
Accuracy:  0.658
Precision:  0.658
Recall:  0.658
F1-score: 0.658



In [20]:
text_svm = Pipeline([
                     ('tfidf', TfidfVectorizer()),
                     ('svm', svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto'))
                     ])
 
text_svm.fit(X_train, y_train)

print('TF Idf + SVM:')
print()
predicted_svm = text_svm.predict(X_test)
metrics_result(y_test,predicted_svm)
print()

TF Idf + SVM:

Metrics:
Accuracy:  0.678
Precision:  0.680
Recall:  0.678
F1-score: 0.678



In [22]:
text_knn = Pipeline([
                     ('tfidf', TfidfVectorizer()),
                     ('knn', KNeighborsClassifier(n_neighbors=7))
                     ])
 
text_knn.fit(X_train, y_train)

print('TF Idf + Knn:')
print()
predicted_knn = text_knn.predict(X_test)
metrics_result(y_test,predicted_knn)
print()

TF Idf + Knn:

Metrics:
Accuracy:  0.586
Precision:  0.610
Recall:  0.586
F1-score: 0.561



In [ ]:
k_range = list(range(1, 10))
param_grid = dict(n_neighbors=k_range)
knn = KNeighborsClassifier()

text_knn_gr = Pipeline([
                     ('tfidf', TfidfVectorizer()),
                     ('knn_gr', GridSearchCV(knn, param_grid, cv=10, scoring='accuracy', return_train_score=False,verbose=1))
                     ])
 
text_knn_gr.fit(X_train, y_train)

print('TF Idf + Knn + Grid Search:')
print()
predicted_knn_gr = text_knn_gr.predict(X_test)
metrics_result(y_test,predicted_knn_gr)
print()

In [32]:
text_gb = Pipeline([
                     ('tfidf', TfidfVectorizer()),
                     ('gb', GradientBoostingClassifier())
                     ])
 
text_gb.fit(X_train, y_train)

print('TF Idf + Gradient Boosting:')
print()
predicted_gb = text_gb.predict(X_test)
metrics_result(y_test,predicted_gb)
print()

TF Idf + Gradient Boosting:

Metrics:
Accuracy:  0.616
Precision:  0.657
Recall:  0.616
F1-score: 0.589



In [33]:
text_sgd = Pipeline([
                     ('tfidf', TfidfVectorizer()),
                     ('sgd', SGDClassifier())
                     ])
 
text_sgd.fit(X_train, y_train)

print('TF Idf + SGD:')
print()
predicted_sgd = text_sgd.predict(X_test)
metrics_result(y_test,predicted_sgd)
print()

TF Idf + SGD:

Metrics:
Accuracy:  0.653
Precision:  0.675
Recall:  0.653
F1-score: 0.642



In [34]:
text_dt = Pipeline([
                     ('tfidf', TfidfVectorizer()),
                     ('dt', DecisionTreeClassifier())
                     ])
 
text_dt.fit(X_train, y_train)

print('TF Idf + Decision Tree:')
print()
predicted_dt = text_dt.predict(X_test)
metrics_result(y_test,predicted_dt)
print()

TF Idf + Decision Tree:

Metrics:
Accuracy:  0.633
Precision:  0.633
Recall:  0.633
F1-score: 0.633



In [35]:
text_ab = Pipeline([
                     ('tfidf', TfidfVectorizer()),
                     ('ab', AdaBoostClassifier())
                     ])
 
text_ab.fit(X_train, y_train)

print('TF Idf + Ada Boost:')
print()
predicted_ab = text_ab.predict(X_test)
metrics_result(y_test,predicted_ab)
print()

TF Idf + Ada Boost:

Metrics:
Accuracy:  0.613
Precision:  0.641
Recall:  0.613
F1-score: 0.593



In [24]:
from gensim.models import Word2Vec

sentences = [sentence.split() for sentence in X_train]
w2v_model = Word2Vec(sentences, window=5, min_count=5, workers=4)

def vectorize(sentence):
    words = sentence.split()
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

X_train_w2v = np.array([vectorize(sentence) for sentence in X_train])
X_test_w2v = np.array([vectorize(sentence) for sentence in X_test])

In [53]:
clf = RandomForestClassifier()
 
clf.fit(X_train_w2v, y_train)
print('Word2Vec + Random Forest:')
print()
predicted = clf.predict(X_test_w2v)
metrics_result(y_test, predicted)
print()

Word2Vec + Random Forest:

Metrics:
Accuracy:  0.673
Precision:  0.673
Recall:  0.673
F1-score: 0.673



In [54]:
lgr = LogisticRegression()
 
lgr.fit(X_train_w2v, y_train)
print('Word2Vec + LogisticRegression:')
print()
predicted = lgr.predict(X_test_w2v)
metrics_result(y_test, predicted)
print()

Word2Vec + LogisticRegression:

Metrics:
Accuracy:  0.646
Precision:  0.647
Recall:  0.646
F1-score: 0.646



In [ ]:
svmm = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
 
svmm.fit(X_train_w2v, y_train)
print('Word2Vec + SVM:')
print()
predicted = svmm.predict(X_test_w2v)
metrics_result(y_test, predicted)
print()

In [25]:
sgd = SGDClassifier()
 
sgd.fit(X_train_w2v, y_train)
print('Word2Vec + SGD:')
print()
predicted = sgd.predict(X_test_w2v)
metrics_result(y_test, predicted)
print()

Word2Vec + SGD:

Metrics:
Accuracy:  0.544
Precision:  0.579
Recall:  0.544
F1-score: 0.490

